In [1]:
import numpy as np
import pickle
import chars2vec
import json

In [7]:
with open('data/2k_references',encoding='UTF-8') as file:
  filedata=json.load(file)

In [8]:
unique_auth = set()
for i in filedata:
  unique_auth.add(i['Author'])
  for j in i['coauthor']:
    unique_auth.add(j)

index2auth={}
auth2index={}
i=0
for auth in unique_auth:
    index2auth[i]=auth
    auth2index[auth]=i
    i+=1

In [9]:
with open('data/auth2index.pickle','wb') as file:
  pickle.dump(auth2index,file)
with open('data/index2auth.pickle','wb') as file:
  pickle.dump(index2auth,file)

In [10]:
def create_styles(strng):
    strng = ''.join([i for i in strng if not i.isdigit()])
    str1=strng.strip()
    str_split=str1.split(" ")
    names=[]
    if len(str_split)==1:
        names.append(str_split)
        
    elif len(str_split)==2:
        style1= str1
        style2=str_split[0][0] + " " + str_split[-1] 
        style3= str_split[-1][0]+ " "+ str_split[0]
        name_list=(style1, style2,style3)
        names.append(name_list)
    
    else:
        style1= str1
        style2=str_split[0][0]+" "+str_split[1][0]+" " + str_split[-1] 
        style3= str_split[-1]+" " +str_split[0][0]+""
        style4= str_split[-1]+ " " +str_split[0][0] + " " + str_split[1]
        name_list=(style1,style2,style3,style4)
        names.append(name_list)

    return names

In [19]:
def generateTrainingDataFromReference(references):

  features_names_list = []
  labels_names_list = []
  tagged_reference_list = []
  reference_number = 0
  combination_number = 0

  with open('data/auth2index.pickle','rb') as file:
    unique_auth= pickle.load(file)

  with open('data/venue_fullforms','rb') as file:
    venue_fullforms= pickle.load(file)
    
  for ref in references:

    ref["ref_number"] = reference_number
    reference_number += 1 
    tagged_reference_list.append(ref)

    try:
      authorList=[]
      authorList.append(ref['Author'])
      for other_auth in ref['coauthor']:
        authorList.append(other_auth)
      venue_fullform=venue_fullforms[ref["journal"]]


      for auth in authorList:
        auth_index= unique_auth[auth]
        coauthors=[]
        coauthors=[x for x in authorList if x!=auth]

        for coauth in coauthors:
          for coauth_style in create_styles(coauth)[0]:
            for auth_style in create_styles(auth)[0]:

              feature_name_dict = {}
              feature_name_dict["ref_number"] = reference_number - 1
              feature_name_dict["Combo_number"] = combination_number
              feature_name_dict["Author_style"] = auth_style
              feature_name_dict["coauthor"] = coauth_style
              feature_name_dict["Title"] = ref["Title"]
              feature_name_dict["journal"] = ref["journal"]
              feature_name_dict["venue_fullform"] = venue_fullform
              features_names_list.append(feature_name_dict)

              label_name_dict = {}
              label_name_dict["ref_number"] = reference_number - 1
              label_name_dict["Combo_number"] = combination_number
              label_name_dict["Author_label"] = auth
              labels_names_list.append(label_name_dict)

              combination_number += 1

    except:
      print(ref)
      continue       
  

  with open('data/features_names_list.pickle','wb') as file:
    pickle.dump(features_names_list,file)
  with open('data/labels_names_list.pickle','wb') as file:
    pickle.dump(labels_names_list,file)
  with open('data/tagged_reference_list.pickle','wb') as file:
    pickle.dump(tagged_reference_list,file)

  print("Task Completed")

In [20]:
generateTrainingDataFromReference(filedata)

Task Completed
